# 🚀 Customize and Deploy `microsoft/phi-4` on Amazon SageMaker AI
---
In this notebook, we explore **Phi-4**, Microsoft's latest and most advanced model in the Phi series. You'll learn how to fine-tune this cutting-edge model, evaluate its exceptional reasoning capabilities, and deploy it using SageMaker for production workloads.

**What is Phi-4?**

Microsoft's **Phi-4** represents a significant leap forward in the Phi model family, delivering state-of-the-art performance across reasoning, mathematics, coding, and instruction-following tasks. Built with advanced training techniques and high-quality data curation, Phi-4 achieves remarkable capabilities while maintaining efficiency.  
🔗 Model card: [microsoft/phi-4 on Hugging Face](https://huggingface.co/microsoft/phi-4)

---

**Key Specifications**

| Feature | Details |
|---|---|
| **Parameters** | ~14 billion (estimated) |
| **Architecture** | Advanced Transformer with optimized attention and MLP layers |
| **Context Length** | Extended context window for complex reasoning |
| **Training Data** | Curated high-quality datasets with advanced filtering |
| **Modalities** | Text-in / Text-out |
| **License** | MIT License |
| **Specialization** | Reasoning, mathematics, coding, and complex problem-solving |

---

**Benchmarks & Behavior**

- Phi-4 achieves **exceptional performance on reasoning benchmarks**, often matching or exceeding much larger models.  
- Outstanding capabilities in **mathematical reasoning, competitive programming**, and complex logical inference.  
- Strong **code generation and debugging** abilities across multiple programming languages.  
- Excellent **instruction following** with nuanced understanding of complex requests.  

---

**Using This Notebook**

Here's what you'll cover:

* Load a sample dataset from Hugging Face and prepare it for fine-tuning  
* Fine-tune with SageMaker Training Jobs using advanced configurations  
* Run Model Evaluation on reasoning and coding benchmarks  
* Deploy to SageMaker Endpoints for high-performance inference  

---

Let's begin by exploring `microsoft/phi-4` and testing its advanced reasoning capabilities.


In [1]:
%pip install -Uq sagemaker datasets

In [2]:
import boto3
import sagemaker

In [3]:
region = boto3.Session().region_name

sess = sagemaker.Session(boto3.Session(region_name=region))

sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

In [4]:
print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")